## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/real_estate_price_size_year_view.csv"
file_type = "csv"

df = spark.read.csv(file_location, header=True, inferSchema=True)

In [0]:
df.printSchema()

root
 |-- price: double (nullable = true)
 |-- size: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- view: string (nullable = true)



In [0]:
df.show()

+----------+-------+----+-----------+
|     price|   size|year|       view|
+----------+-------+----+-----------+
|234314.144| 643.09|2015|No sea view|
|228581.528| 656.22|2009|No sea view|
|281626.336| 487.29|2018|   Sea view|
|401255.608|1504.75|2015|No sea view|
|458674.256|1275.46|2009|   Sea view|
| 245050.28| 575.19|2006|   Sea view|
|265129.064| 570.89|2015|   Sea view|
| 175716.48| 620.82|2006|No sea view|
|331101.344| 682.26|2018|   Sea view|
|218630.608| 694.52|2009|No sea view|
|279555.096|1060.36|2009|No sea view|
|494778.992|1842.51|2009|   Sea view|
|215472.104| 694.52|2015|No sea view|
|418753.008|1009.25|2018|No sea view|
|444192.008|1300.96|2006|   Sea view|
|440201.616|1379.72|2006|   Sea view|
|  248337.6| 690.54|2018|No sea view|
| 234178.16| 623.94|2006|   Sea view|
|225451.984| 681.07|2006|No sea view|
|299416.976|1027.76|2018|No sea view|
+----------+-------+----+-----------+
only showing top 20 rows



In [0]:
df.select('price').show()

+----------+
|     price|
+----------+
|234314.144|
|228581.528|
|281626.336|
|401255.608|
|458674.256|
| 245050.28|
|265129.064|
| 175716.48|
|331101.344|
|218630.608|
|279555.096|
|494778.992|
|215472.104|
|418753.008|
|444192.008|
|440201.616|
|  248337.6|
| 234178.16|
|225451.984|
|299416.976|
+----------+
only showing top 20 rows



In [0]:
df.columns

Out[9]: ['price', 'size', 'year', 'view']

In [0]:
## Handling categorical features
from pyspark.ml.feature import StringIndexer

In [0]:
indexer = StringIndexer(inputCol='view', outputCol='view_indexed')
df1 = indexer.fit(df).transform(df)
df1.show()

+----------+-------+----+-----------+------------+
|     price|   size|year|       view|view_indexed|
+----------+-------+----+-----------+------------+
|234314.144| 643.09|2015|No sea view|         0.0|
|228581.528| 656.22|2009|No sea view|         0.0|
|281626.336| 487.29|2018|   Sea view|         1.0|
|401255.608|1504.75|2015|No sea view|         0.0|
|458674.256|1275.46|2009|   Sea view|         1.0|
| 245050.28| 575.19|2006|   Sea view|         1.0|
|265129.064| 570.89|2015|   Sea view|         1.0|
| 175716.48| 620.82|2006|No sea view|         0.0|
|331101.344| 682.26|2018|   Sea view|         1.0|
|218630.608| 694.52|2009|No sea view|         0.0|
|279555.096|1060.36|2009|No sea view|         0.0|
|494778.992|1842.51|2009|   Sea view|         1.0|
|215472.104| 694.52|2015|No sea view|         0.0|
|418753.008|1009.25|2018|No sea view|         0.0|
|444192.008|1300.96|2006|   Sea view|         1.0|
|440201.616|1379.72|2006|   Sea view|         1.0|
|  248337.6| 690.54|2018|No sea

In [0]:
df1.columns

Out[12]: ['price', 'size', 'year', 'view', 'view_indexed']

In [0]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=['size', 'view_indexed'], outputCol='Independent Features')

In [0]:
output = featureassembler.transform(df1)

In [0]:
output.show()

+----------+-------+----+-----------+------------+--------------------+
|     price|   size|year|       view|view_indexed|Independent Features|
+----------+-------+----+-----------+------------+--------------------+
|234314.144| 643.09|2015|No sea view|         0.0|        [643.09,0.0]|
|228581.528| 656.22|2009|No sea view|         0.0|        [656.22,0.0]|
|281626.336| 487.29|2018|   Sea view|         1.0|        [487.29,1.0]|
|401255.608|1504.75|2015|No sea view|         0.0|       [1504.75,0.0]|
|458674.256|1275.46|2009|   Sea view|         1.0|       [1275.46,1.0]|
| 245050.28| 575.19|2006|   Sea view|         1.0|        [575.19,1.0]|
|265129.064| 570.89|2015|   Sea view|         1.0|        [570.89,1.0]|
| 175716.48| 620.82|2006|No sea view|         0.0|        [620.82,0.0]|
|331101.344| 682.26|2018|   Sea view|         1.0|        [682.26,1.0]|
|218630.608| 694.52|2009|No sea view|         0.0|        [694.52,0.0]|
|279555.096|1060.36|2009|No sea view|         0.0|       [1060.3

In [0]:
output.select('Independent Features').show()

+--------------------+
|Independent Features|
+--------------------+
|        [643.09,0.0]|
|        [656.22,0.0]|
|        [487.29,1.0]|
|       [1504.75,0.0]|
|       [1275.46,1.0]|
|        [575.19,1.0]|
|        [570.89,1.0]|
|        [620.82,0.0]|
|        [682.26,1.0]|
|        [694.52,0.0]|
|       [1060.36,0.0]|
|       [1842.51,1.0]|
|        [694.52,0.0]|
|       [1009.25,0.0]|
|       [1300.96,1.0]|
|       [1379.72,1.0]|
|        [690.54,0.0]|
|        [623.94,1.0]|
|        [681.07,0.0]|
|       [1027.76,0.0]|
+--------------------+
only showing top 20 rows



In [0]:
finalized_data = output.select('Independent Features', 'price')

In [0]:
finalized_data.show()

+--------------------+----------+
|Independent Features|     price|
+--------------------+----------+
|        [643.09,0.0]|234314.144|
|        [656.22,0.0]|228581.528|
|        [487.29,1.0]|281626.336|
|       [1504.75,0.0]|401255.608|
|       [1275.46,1.0]|458674.256|
|        [575.19,1.0]| 245050.28|
|        [570.89,1.0]|265129.064|
|        [620.82,0.0]| 175716.48|
|        [682.26,1.0]|331101.344|
|        [694.52,0.0]|218630.608|
|       [1060.36,0.0]|279555.096|
|       [1842.51,1.0]|494778.992|
|        [694.52,0.0]|215472.104|
|       [1009.25,0.0]|418753.008|
|       [1300.96,1.0]|444192.008|
|       [1379.72,1.0]|440201.616|
|        [690.54,0.0]|  248337.6|
|        [623.94,1.0]| 234178.16|
|        [681.07,0.0]|225451.984|
|       [1027.76,0.0]|299416.976|
+--------------------+----------+
only showing top 20 rows



In [0]:
from pyspark.ml.regression import LinearRegression
# using train test split
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol= 'Independent Features', labelCol= 'price')
regressor = regressor.fit(train_data)

In [0]:
regressor.coefficients

Out[21]: DenseVector([214.6734, 56377.6139])

In [0]:
regressor.intercept

Out[22]: 81567.86973783036

In [0]:
## prediction
pred_result = regressor.evaluate(test_data)

In [0]:
## final predictions
pred_result.predictions.show()

+--------------------+----------+------------------+
|Independent Features|     price|        prediction|
+--------------------+----------+------------------+
|        [549.69,0.0]| 171795.24|199571.70664918036|
|        [570.89,1.0]|271793.312| 260500.3972708304|
|        [620.71,1.0]| 268125.08|271195.42747862655|
|        [620.82,0.0]| 175716.48| 214841.4276182799|
|        [633.19,0.0]|204302.976|217496.93792880655|
|        [643.09,1.0]|282683.544|275999.81880842336|
|         [647.5,0.0]| 225656.12|220568.91469062032|
|        [648.29,0.0]|225145.248|220738.50669913416|
|        [681.07,1.0]| 297760.44|284153.11562279705|
|        [698.29,1.0]| 300061.48|287849.79206154146|
|        [727.88,1.0]|269523.056|294201.97881081264|
|       [1009.25,1.0]|  355251.2|354604.64138744044|
|       [1021.95,1.0]|334938.872|357330.99392937165|
|       [1021.95,1.0]| 393069.76|357330.99392937165|
|       [1028.41,1.0]|327252.112|358717.78427747206|
|       [1060.36,0.0]|293044.496| 309198.98638

In [0]:
# Performance Metrics
pred_result.r2, pred_result.meanAbsoluteError, pred_result.meanSquaredError

Out[25]: (0.924072178979023, 16953.334342652543, 403059988.0202539)